In [91]:
import sklearn.cross_validation as cv
import pandas as pd
import numpy as np
import sys

In [43]:
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV
from sklearn.metrics import auc, roc_curve

In [2]:
train = pd.read_csv('../Input/train_processed.csv') 
test_raw = pd.read_csv('../Input/test_processed.csv')

NameError: name 'pd' is not defined

In [1]:
train

NameError: name 'train' is not defined

In [46]:
cv.StratifiedKFold
seed = 234523
np.random.seed(seed)
train_idx = np.random.choice(train.index.values, 30000)
val_idx = np.random.choice(np.delete(train.index.values,train_idx), 15000)
train_s = train.ix[train_idx] 
validation_s = train.ix[val_idx] 

In [47]:
cols_features = list(set(train.columns.tolist()) - set(['ID', 'target']))
cols_target = ['target']

In [99]:
models = [#ExtraTreesClassifier(random_state=seed),
          #RandomForestClassifier(random_state=seed),
          #LogisticRegressionCV(),
          #RidgeClassifierCV(),
          GradientBoostingClassifier(random_state=seed, verbose=1)]

In [80]:
X_train = train_s[cols_features]
y_train = np.ravel(train_s[cols_target])
X_val = validation_s[cols_features]
y_val = np.ravel(validation_s[cols_target])

In [ ]:
results = []
for model in models:
    print_time(model)
    sys.stdout.flush()
    model.fit(X_train, y_train)
    pred_train = model.predict_proba(X_train)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_train, pred_train, pos_label=1)
    auc_train = auc(fpr, tpr)
    pred_val = model.predict_proba(X_val)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_val, pred_val, pos_label=1)
    auc_val = auc(fpr, tpr)
    results.append(dict(
          pred_train = pred_train,
          auc_train = auc_train,
          pred_val = pred_val,
          auc_val = auc_val,
          model = model
        ))
    
    print_time('Train auc %s' %auc_train)
    print_time('Val auc %s' %auc_val)
    sys.stdout.flush()

15:02:27 10/13/15 BRT - ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=234523, verbose=0,
           warm_start=False)
15:02:35 10/13/15 BRT - Train auc 1.0
15:02:35 10/13/15 BRT - Val auc 0.687078837884
15:02:35 10/13/15 BRT - RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=234523, verbose=0,
            warm_start=False)
15:02:44 10/13/15 BRT - Train auc 0.99947934142
15:02:44 10/13/15 BRT - Val auc 0.691080118392
15:02:44 10/13/15 BRT - LogisticRegressionCV(Cs=10, class_we

In [ ]:
sorted(results.items(),
      key=lambda x: x[1]['auc_test'],
      reverse=True)

In [97]:
import time
def print_time(msg):
    '''
    Print the message with a time before iter
    '''
    print '%s - %s' % (time.strftime('%X %x %Z'), msg)